In [1]:
!pip install torch-geometric -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00


## Imports

In [2]:
import numpy as np
import torch
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import Data
import networkx as nx
from torch_geometric.utils import from_networkx

## Graph Robustness Metrics

**1. Effective Graph Resistance (EGR)**  
   $$
   R_g = \frac{2}{N-1} \sum_{i=1}^{N-c} \frac{1}{\lambda_i}
   $$
   where $ \lambda_i $ are the eigenvalues of the Laplacian matrix of the graph.

In [3]:
def compute_effective_resistance(graph):
    laplacian = nx.laplacian_matrix(graph).toarray()
    eigenvalues = np.linalg.eigvalsh(laplacian)
    eigenvalues = eigenvalues[eigenvalues > 1e-8]  # Avoid zero eigenvalues
    N = graph.number_of_nodes()
    return (2 / (N - 1)) * np.sum(1 / eigenvalues)

**2. Weighted Spectrum (WS)**  
   $$
   W_s = \sum_i (1 - \lambda_i)^n
   $$
   where $ n $ controls the depth of analysis

In [4]:
def compute_weighted_spectrum(graph, n=4):
    laplacian = nx.normalized_laplacian_matrix(graph).toarray()
    eigenvalues = np.linalg.eigvalsh(laplacian)
    return np.sum((1 - eigenvalues) ** n)

## Algorithm 1: ILGR Embedding Module
**Input:** Graph $ G $, input node features $ X_v $ $ \forall v \in V $, unknown model weights $ W $ (combination weights) and $ Q $ (aggregation weights).

**Output:** Nodes embedding vector $ z_v $ $ \forall v \in V $.

**1. Initialize**: $ h^0_v = X_v $ for all $ v \in V $.
**2. For each layer** $ l = 1 $ to $ L $ do:
   - For each node $ v = 1 $ to $ V $:
     1. Compute neighborhood embedding using attention mechanism:
        $$
        h^l_{N(v)} = \text{Attention}(Q^l h^{l-1}_k) \quad \forall k \in N(v)
        $$
     2. Compute new embedding for node $ v $ using a **skip connection**:
        $$
        h^l_v = \text{ReLU} \left( W^l \left[ h^{l-1}_v || h^{l-2}_v || h^l_{N(v)} \right] \right)
        $$
**3. Return**: Final embedding vector $ z_v = h^L_v $ for all $ v \in V $.


In [5]:
class ILGRNodeEmbedding(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(1, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.attention = torch.nn.MultiheadAttention(hidden_channels, 1)
        
    def forward(self, x, edge_index):
        # Skip connections and attention
        h1 = self.conv1(x, edge_index)
        h2 = self.conv2(h1, edge_index)
        h, _ = self.attention(h2, h2, h2)
        return torch.cat([h1, h2, h], dim=-1)

## Regression Module

The regression module applies a **non-linear transformation** using multiple layers:

$$
y_m = f(W_m \cdot y_{m-1} + b_m)
$$

where:
- $ y_m $ is the output of the $ m^{th} $ layer.
- $ W_m $ and $ b_m $ are the **weights** and **biases** of the $ m^{th} $ layer.
- $ f $ is an **activation function**
- The input to the first layer is the **node embedding**:



In [6]:
class RegressionModule(torch.nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, 12)
        self.fc2 = torch.nn.Linear(12, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

## Full Model

In [7]:
class ILGR(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.embedding = ILGRNodeEmbedding(hidden_channels)
        self.regression = RegressionModule(hidden_channels * 3)  # Concatenated embeddings
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        embedding = self.embedding(x, edge_index)
        return self.regression(embedding)

## Algorithm 2: ILGR Training
**Input:** Model with unknown weights.
**Output:** Trained model.

1. Compute ground truth **criticality scores** of nodes based on graph robustness score.
2. **For each epoch do**:
   - Get each **node embedding** from the embedding module.
   - Estimate **criticality scores** of nodes/links using the regression module.
   - Update weights of both modules by solving the loss function:
     $$
     L_{ij} = -f(r_{ij}) \log(σ(\hat{y}_{ij})) - (1 - f(r_{ij})) \log(1 - σ(\hat{y}_{ij}))
     $$
3. **End loop**.
4. Predict nodescores on the test graph.
5. **Return**: Top $ N\% $ of most critical nodes.


### Criticality Score Calculation

### Algorithm 3: Conventional Approach for Identifying Critical Nodes/Links
**Input:** Graph $ G $ with $ V $ nodes.
**Output:** Node critical scores.

**1. For each node/link** $ n $ in $ V $:
   - Remove node $ n $ from the graph $ G $.
   - Compute robustness metric of the **residual graph** $ (G - n) $.
   - Assign a **criticality score** to node $ n $.

**2. End loop**.

3. Rank nodes based on computed **criticality scores**.
4. Top ranks correspond to the **most critical nodes**.
**5. Return**: Top $ N\% $ of most critical nodes.

In [8]:
def compute_criticality_scores(graph, metric):
    scores = []
    for node in graph.nodes():
        subgraph = graph.copy()
        subgraph.remove_node(node)
        score = metric(graph) - metric(subgraph)  # Drop in robustness
        scores.append(score)
    return scores

### Ranking Loss
$$
     L_{ij} = -f(r_{ij}) \log(σ(\hat{y}_{ij})) - (1 - f(r_{ij})) \log(1 - σ(\hat{y}_{ij}))
     $$

In [9]:
def pairwise_ranking_loss(y_pred, y_true):
    # Compare all pairs of nodes
    loss = 0
    for i in range(len(y_true)):
        for j in range(i+1, len(y_true)):
            if y_true[i] > y_true[j]:
                loss += torch.log(torch.sigmoid(y_pred[i] - y_pred[j]))
            else:
                loss += torch.log(torch.sigmoid(y_pred[j] - y_pred[i]))
    return -loss / (len(y_true) * (len(y_true)-1) / 2)

## Graph Preprocessing

### Generate Synthetic Graphs

In [10]:
# Power-law graph (Barabási-Albert model)
def generate_power_law(n, m):
    return nx.barabasi_albert_graph(n, m)

# Power-law cluster graph (Holme-Kim model)
def generate_power_law_cluster(n, m, p):
    return nx.powerlaw_cluster_graph(n, m, p)

### real-world datasets (load function)

In [11]:
def load_real_world_graph(dataset_name):
    G = nx.read_edgelist(dataset_name, nodetype=int)
    return G

### Convert NetworkX graph to PyTorch Geometric format

In [12]:
def nx_to_pyg(nx_graph):
    # Convert the NetworkX graph to PyG data format
    pyg_data = from_networkx(nx_graph)
    
    # Ensure the attributes are tensors (optional, based on your needs)
    pyg_data.x = torch.tensor(pyg_data.x, dtype=torch.float) if pyg_data.x is not None else None
    pyg_data.edge_index = torch.tensor(pyg_data.edge_index, dtype=torch.long)
    pyg_data.edge_attr = torch.tensor(pyg_data.edge_attr, dtype=torch.float) if pyg_data.edge_attr is not None else None
    
    return pyg_data

### Training Loop

In [13]:
def train_model(model, data, y_true, epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        y_pred = model(data)

        # Compute loss
        loss = pairwise_ranking_loss(y_pred, y_true)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
